In [1]:
import pandas as pd
import numpy as np

In [2]:
train = pd.read_csv('./train.csv')
test = pd.read_csv('./test.csv')
stores = pd.read_csv('./stores.csv')
transactions = pd.read_csv('./transactions.csv')
holidays = pd.read_csv('./holidays_events.csv')
oil = pd.read_csv('./oil.csv')

In [3]:
oil['dcoilwtico'] = oil['dcoilwtico'].interpolate()

In [4]:
oil['dcoilwtico'] = oil['dcoilwtico'].fillna(93.14)

In [5]:
date_range = pd.date_range(start=oil['date'].min(), end=oil['date'].max())
df_dates = pd.DataFrame(date_range, columns=['date'])

In [6]:
oil['date'] = pd.to_datetime(oil['date'])

In [7]:
oil = pd.merge(df_dates, oil, on='date', how='left')

In [8]:
# Интерполяция
oil['dcoilwtico'].interpolate(method='linear', inplace=True)

In [9]:
oil_train = oil[oil['date'] <= '2017-08-15']
oil_test = oil[oil['date'] > '2017-08-15']

In [10]:
oil_train

,date,dcoilwtico
0,2013-01-01,93.140000
1,2013-01-02,93.140000
2,2013-01-03,92.970000
3,2013-01-04,93.120000
4,2013-01-05,93.146667
...,...,...
1683,2017-08-11,48.810000
1684,2017-08-12,48.403333
1685,2017-08-13,47.996667
1686,2017-08-14,47.590000


In [11]:
holidays

,date,type,locale,locale_name,description,transferred
0,2012-03-02,Holiday,Local,Manta,Fundacion de Manta,False
1,2012-04-01,Holiday,Regional,Cotopaxi,Provincializacion de Cotopaxi,False
2,2012-04-12,Holiday,Local,Cuenca,Fundacion de Cuenca,False
3,2012-04-14,Holiday,Local,Libertad,Cantonizacion de Libertad,False
4,2012-04-21,Holiday,Local,Riobamba,Cantonizacion de Riobamba,False
...,...,...,...,...,...,...
345,2017-12-22,Additional,National,Ecuador,Navidad-3,False
346,2017-12-23,Additional,National,Ecuador,Navidad-2,False
347,2017-12-24,Additional,National,Ecuador,Navidad-1,False
348,2017-12-25,Holiday,National,Ecuador,Navidad,False


In [12]:
holidays_dict = {}

for index, row in holidays.iterrows():
    if row['type'] == 'Transfer':
        # Записываем настоящие даты празднования для переносных праздников
        holidays_dict[row['date']] = (row['locale'], row['locale_name'])
    else:
        # Записываем обычные праздники
        holidays_dict[row['date']] = (row['locale'], row['locale_name'])

In [13]:
merged_train = train.merge(stores, on='store_nbr', how='left')

In [14]:
merged_train

,id,date,store_nbr,family,sales,onpromotion,city,state,type,cluster
0,0,2013-01-01,1,AUTOMOTIVE,0.000,0,Quito,Pichincha,D,13
1,1,2013-01-01,1,BABY CARE,0.000,0,Quito,Pichincha,D,13
2,2,2013-01-01,1,BEAUTY,0.000,0,Quito,Pichincha,D,13
3,3,2013-01-01,1,BEVERAGES,0.000,0,Quito,Pichincha,D,13
4,4,2013-01-01,1,BOOKS,0.000,0,Quito,Pichincha,D,13
...,...,...,...,...,...,...,...,...,...,...
3000883,3000883,2017-08-15,9,POULTRY,438.133,0,Quito,Pichincha,B,6
3000884,3000884,2017-08-15,9,PREPARED FOODS,154.553,1,Quito,Pichincha,B,6
3000885,3000885,2017-08-15,9,PRODUCE,2419.729,148,Quito,Pichincha,B,6
3000886,3000886,2017-08-15,9,SCHOOL AND OFFICE SUPPLIES,121.000,8,Quito,Pichincha,B,6


In [15]:
def check_holiday(row):
    if row['date'] in holidays_dict:
        locale, locale_name = holidays_dict[row['date']]

        # Если праздник национальный, то он отмечается везде
        if locale == 'National':
            return True

        # Если праздник региональный, учитываем штат
        if locale == 'Regional' and row['state'] == locale_name:
            return True

        # Если праздник местный, учитываем город
        if locale == 'Local' and row['city'] == locale_name:
            return True

    return False

merged_train['isHoliday'] = merged_train.apply(check_holiday, axis=1)

In [16]:
merged_train['isHoliday'].value_counts()

False    2739099
True      261789
Name: isHoliday, dtype: int64

In [17]:
merged_train['date'] = pd.to_datetime(merged_train['date'])

In [18]:
merged_train = merged_train.merge(oil_train, on='date', how='left')

In [19]:
merged_train.isna().sum()

id             0
date           0
store_nbr      0
family         0
sales          0
onpromotion    0
city           0
state          0
type           0
cluster        0
isHoliday      0
dcoilwtico     0
dtype: int64

In [20]:
merged_train

,id,date,store_nbr,family,sales,onpromotion,city,state,type,cluster,isHoliday,dcoilwtico
0,0,2013-01-01,1,AUTOMOTIVE,0.000,0,Quito,Pichincha,D,13,True,93.14
1,1,2013-01-01,1,BABY CARE,0.000,0,Quito,Pichincha,D,13,True,93.14
2,2,2013-01-01,1,BEAUTY,0.000,0,Quito,Pichincha,D,13,True,93.14
3,3,2013-01-01,1,BEVERAGES,0.000,0,Quito,Pichincha,D,13,True,93.14
4,4,2013-01-01,1,BOOKS,0.000,0,Quito,Pichincha,D,13,True,93.14
...,...,...,...,...,...,...,...,...,...,...,...,...
3000883,3000883,2017-08-15,9,POULTRY,438.133,0,Quito,Pichincha,B,6,False,47.57
3000884,3000884,2017-08-15,9,PREPARED FOODS,154.553,1,Quito,Pichincha,B,6,False,47.57
3000885,3000885,2017-08-15,9,PRODUCE,2419.729,148,Quito,Pichincha,B,6,False,47.57
3000886,3000886,2017-08-15,9,SCHOOL AND OFFICE SUPPLIES,121.000,8,Quito,Pichincha,B,6,False,47.57


In [21]:
from sklearn.model_selection import train_test_split

X = merged_train.drop(columns=['date', 'sales'])
y = merged_train['sales']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [22]:
from catboost import CatBoostRegressor

categorical_features = ['store_nbr', 'family', 'city', 'state', 'type', 'cluster']

# model = CatBoostRegressor(iterations=1000, depth=10, learning_rate=0.05, loss_function='RMSE')
# model.fit(X_train, y_train, cat_features=categorical_features, eval_set=(X_test, y_test), verbose=100, early_stopping_rounds=10)

In [23]:
# y_pred = model.predict(X_test)
from sklearn.metrics import mean_squared_error
# print('The RMSE of prediction is:', mean_squared_error(y_test, y_pred) ** 0.5)

In [24]:
merged_test = test.merge(stores, on='store_nbr', how='left')

In [25]:
merged_test['isHoliday'] = merged_test.apply(check_holiday, axis=1)

In [26]:
merged_test.isna().sum()

id             0
date           0
store_nbr      0
family         0
onpromotion    0
city           0
state          0
type           0
cluster        0
isHoliday      0
dtype: int64

In [27]:
merged_test['date'] = pd.to_datetime(merged_test['date'])

In [28]:
merged_test = merged_test.merge(oil_test, on='date', how='left')

In [29]:
merged_test.isna().sum()

id             0
date           0
store_nbr      0
family         0
onpromotion    0
city           0
state          0
type           0
cluster        0
isHoliday      0
dcoilwtico     0
dtype: int64

In [30]:
merged_test

,id,date,store_nbr,family,onpromotion,city,state,type,cluster,isHoliday,dcoilwtico
0,3000888,2017-08-16,1,AUTOMOTIVE,0,Quito,Pichincha,D,13,False,46.80
1,3000889,2017-08-16,1,BABY CARE,0,Quito,Pichincha,D,13,False,46.80
2,3000890,2017-08-16,1,BEAUTY,2,Quito,Pichincha,D,13,False,46.80
3,3000891,2017-08-16,1,BEVERAGES,20,Quito,Pichincha,D,13,False,46.80
4,3000892,2017-08-16,1,BOOKS,0,Quito,Pichincha,D,13,False,46.80
...,...,...,...,...,...,...,...,...,...,...,...
28507,3029395,2017-08-31,9,POULTRY,1,Quito,Pichincha,B,6,False,47.26
28508,3029396,2017-08-31,9,PREPARED FOODS,0,Quito,Pichincha,B,6,False,47.26
28509,3029397,2017-08-31,9,PRODUCE,1,Quito,Pichincha,B,6,False,47.26
28510,3029398,2017-08-31,9,SCHOOL AND OFFICE SUPPLIES,9,Quito,Pichincha,B,6,False,47.26


In [31]:
merged_train

,id,date,store_nbr,family,sales,onpromotion,city,state,type,cluster,isHoliday,dcoilwtico
0,0,2013-01-01,1,AUTOMOTIVE,0.000,0,Quito,Pichincha,D,13,True,93.14
1,1,2013-01-01,1,BABY CARE,0.000,0,Quito,Pichincha,D,13,True,93.14
2,2,2013-01-01,1,BEAUTY,0.000,0,Quito,Pichincha,D,13,True,93.14
3,3,2013-01-01,1,BEVERAGES,0.000,0,Quito,Pichincha,D,13,True,93.14
4,4,2013-01-01,1,BOOKS,0.000,0,Quito,Pichincha,D,13,True,93.14
...,...,...,...,...,...,...,...,...,...,...,...,...
3000883,3000883,2017-08-15,9,POULTRY,438.133,0,Quito,Pichincha,B,6,False,47.57
3000884,3000884,2017-08-15,9,PREPARED FOODS,154.553,1,Quito,Pichincha,B,6,False,47.57
3000885,3000885,2017-08-15,9,PRODUCE,2419.729,148,Quito,Pichincha,B,6,False,47.57
3000886,3000886,2017-08-15,9,SCHOOL AND OFFICE SUPPLIES,121.000,8,Quito,Pichincha,B,6,False,47.57


In [32]:
test_end = merged_test.drop(columns=['date'])

In [106]:
# new_data_predictions = model.predict(test_end)

In [107]:
# new_data_predictions

array([ -14.57297712,  -17.62038046,  -16.74172151, ..., 1465.28239896,
         53.7785436 ,    3.12809421])

In [111]:
# sub_1 = pd.DataFrame({
#     'id': test_end['id'].values,
#     'sales': new_data_predictions
# })

In [113]:
# sub_1['sales'] = sub_1['sales'].apply(lambda x: max(0, x))

In [114]:
# sub_1

,id,sales
0,3000888,0.000000
1,3000889,0.000000
2,3000890,0.000000
3,3000891,2357.486769
4,3000892,0.000000
...,...,...
28507,3029395,277.628270
28508,3029396,76.153093
28509,3029397,1465.282399
28510,3029398,53.778544


In [115]:
# sub_1.to_csv("sub_1.csv", index=False)

In [33]:
y_train_log = np.log1p(y_train)
y_test_log = np.log1p(y_test)

In [63]:
model = CatBoostRegressor(iterations=10000, depth=7, learning_rate=0.005, loss_function='RMSE')
model.fit(X_train, y_train_log, cat_features=categorical_features, eval_set=(X_test, y_test_log), verbose=100, early_stopping_rounds=10)

0:	learn: 2.6851799	test: 2.6870041	best: 2.6870041 (0)	total: 6.39s	remaining: 17h 45m 50s
100:	learn: 1.9725096	test: 1.9730470	best: 1.9730470 (100)	total: 1m 52s	remaining: 3h 4m 31s
200:	learn: 1.5803261	test: 1.5799204	best: 1.5799204 (200)	total: 3m 39s	remaining: 2h 58m 22s
300:	learn: 1.3531976	test: 1.3520845	best: 1.3520845 (300)	total: 5m 29s	remaining: 2h 57m 5s
400:	learn: 1.2267002	test: 1.2246975	best: 1.2246975 (400)	total: 7m 42s	remaining: 3h 4m 27s
500:	learn: 1.1554803	test: 1.1528508	best: 1.1528508 (500)	total: 9m 55s	remaining: 3h 8m 16s
600:	learn: 1.1086653	test: 1.1056402	best: 1.1056402 (600)	total: 12m 21s	remaining: 3h 13m 21s
700:	learn: 1.0809659	test: 1.0777272	best: 1.0777272 (700)	total: 14m 28s	remaining: 3h 11m 58s
800:	learn: 1.0505869	test: 1.0472604	best: 1.0472604 (800)	total: 17m 2s	remaining: 3h 15m 38s
900:	learn: 1.0284942	test: 1.0249873	best: 1.0249873 (900)	total: 19m 25s	remaining: 3h 16m 8s
1000:	learn: 1.0141135	test: 1.0104206	best: 1

In [54]:
y_pred_log = model.predict(test_end)

In [55]:
predictions = np.expm1(y_pred_log)

In [57]:
predictions

array([3.93828970e+00, 1.95890483e-01, 7.22570491e+00, ...,
       1.27791727e+03, 5.86242166e+01, 2.07364475e+01])

In [58]:
sub_2 = pd.DataFrame({
    'id': test_end['id'].values,
    'sales': predictions
})

In [59]:
sub_2

,id,sales
0,3000888,3.938290
1,3000889,0.195890
2,3000890,7.225705
3,3000891,2339.400786
4,3000892,0.195890
...,...,...
28507,3029395,250.260954
28508,3029396,127.788575
28509,3029397,1277.917270
28510,3029398,58.624217


In [60]:
sub_2['sales'] = sub_2['sales'].apply(lambda x: max(0, x))

In [62]:
sub_2.to_csv('sub_3.csv', index=False)